Since |CR_24|=2704156 is too big to do by hand. We can either use Randomization test or do permutation test without random sampling of all possible assignmetns by coding (from scratch or using packages in R like perm, jmuOutlier, or potential other Python packages as well).

Here we try to do it by python coding from scratch to implement the permutation test.

In [51]:
import numpy as np

import scipy
import scipy.stats
from scipy.stats import ks_2samp as ksstats

In [17]:
# The total number of assignments we can do = C(24,12)=

def numAssignment(n):
    n=int(n)
    half=int(n/2)
    nominator=1
    for i in range(n,half,-1):
        nominator=nominator*i

    denominator=1
    for i in range(half,0,-1):
        denominator=denominator*i
    
    return int(nominator/denominator)

numAssignment(24)

2704156

As the exact complete enumeration/permutation assignment is harder to code, we use Monte Carlo to approximate the exact result. But we can set the number of MC to be really big to reduce the potential bias.

Codes referred to:
https://stackoverflow.com/questions/24795535/pythons-implementation-of-permutation-test-with-permutation-number-as-input


In [55]:
def exact_mc_perm_test(xs, ys, nmc,method):
    if method=="mean":
        n, k = len(xs), 0
        tao = (sum(xs) - sum(ys))*2/n
#             same with np.mean(xs)-np.mean(ys)
        zs = np.concatenate([xs, ys])
        for j in range(nmc):
            np.random.shuffle(zs)
            tao2=(sum(zs[:n]) - sum(zs[n:]))*2/n
            k +=  (  np.abs(tao) <= np.abs(tao2) )
            fisher_p=k / nmc
        return fisher_p
    
    if method=="median":
        n, k = len(xs), 0
        tao = np.median(xs) - np.median(ys)
        zs = np.concatenate([xs, ys])
        for j in range(nmc):
            np.random.shuffle(zs)
            tao2 = np.median(zs[:n]) - np.median(zs[n:])
            k +=  (  np.abs(tao) <= np.abs(tao2) )
            fisher_p=k / nmc
        return fisher_p

    if method=="ks":
        n, k = len(xs), 0
        tao = ksstats(xs,ys)[0]
        zs = np.concatenate([xs, ys])
        for j in range(nmc):
            np.random.shuffle(zs)
            tao2 = ksstats(zs[:n],zs[n:])[0]
            k +=  (  np.abs(tao) <= np.abs(tao2) )
            fisher_p=k / nmc
        return fisher_p
    
    
    # we use scipy.stats.ks_2samp to get the value of KS 
# https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.ks_2samp.html


Data input

In [47]:
y_0 = np.array([21.51,28.14,24.04,23.45,23.68,19.79,28.4,20.98,22.51,20.1,26.91,26.25])
y_1 = np.array ([25.71,26.37,22.8,25.34,24.97,28.14,29.58,30.92,34.02,21.9,31.53,20.73])
n=len(y_0)+len(y_1)

### Result

1) The mean difference and p-value for the sharp null

In [39]:
(sum(y_1) - sum(y_0))*2/n

3.0208333333333335

In [40]:
exact_mc_perm_test(y_1,y_0,3000,"mean")

0.050666666666666665

2) The median difference and p-value for it

In [42]:
np.median(y_1) - np.median(y_0)

2.4750000000000014

In [43]:
exact_mc_perm_test(y_1,y_0,3000,"median")

0.24833333333333332

In [ ]:
3) The median difference and p-value for KS 

In [59]:
ksstats(y_1 ,y_0)
# ksstats(y_1 ,y_1)
# Ks_2sampResult(statistic=0.0, pvalue=1.0)

Ks_2sampResult(statistic=0.41666666666666663, pvalue=0.186196839004176)

In [60]:
exact_mc_perm_test(y_1,y_0,3000,"ks")

0.24366666666666667